# FPN ? (확실하지 않음..)

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models

In [38]:
def _upsample_add(x, y):
    _,_,H,W = y.size()
    return functional.interpolate(x, size=(H,W), mode='bilinear') + y

# --------------------------------------------------------------------------------
def normal_init(m, mean, stddev, truncated=False):
    # x is a parameter
    if truncated:
        m.weight.data.normal_().fmod_(2).mul_(stddev).add_(mean)  # not a perfect approximation
    else:
        m.weight.data.normal_(mean, stddev)
        m.bias.data.zero_()


# --------------------------------------------------------------------------------
def fpn(out_channels,C2,C3,C4):
      
#         P5 = torch.nn.MaxPool2d(kernel_size=1, stride=2, padding=0)
        P5 = torch.nn.AdaptiveAvgPool2d(1)

        P4_conv1 = torch.nn.Conv2d(512, out_channels, kernel_size=1, stride=1, padding=0)
        P4_conv2 = torch.nn.Conv2d(out_channels, out_channels, 3, 1, 1)

        P3_conv1 = torch.nn.Conv2d(512, out_channels, kernel_size=1, stride=1, padding=0)
        P3_conv2 = torch.nn.Conv2d(out_channels, out_channels, 3, 1, 1)

        P2_conv1 = torch.nn.Conv2d(256, out_channels, kernel_size=1, stride=1, padding=0)
        P2_conv2 = torch.nn.Conv2d(out_channels, out_channels, 3, 1, 1)

        normal_init(P4_conv1, 0, 0.01)
        normal_init(P4_conv2, 0, 0.01)
        normal_init(P3_conv1, 0, 0.01)
        normal_init(P3_conv2, 0, 0.01)
        normal_init(P2_conv1, 0, 0.01)
        normal_init(P2_conv2, 0, 0.01)




        p4_out = P4_conv1(C4)

        p5_out = P5(p4_out)

        p3_out = _upsample_add(p4_out, P3_conv1(C3))
        p2_out = _upsample_add(p3_out, P2_conv1(C2))

        p4_out = P4_conv2(p4_out)
        p3_out = P3_conv2(p3_out)
        p2_out = P2_conv2(p2_out)

        return p2_out, p3_out, p4_out, p5_out

In [32]:
from efficientnet_pytorch import EfficientNet


# CNN (basic)

In [33]:
# from tensorflow.python.keras.models import Sequential
# from tensorflow.python.keras.layers.normalization import BatchNormalization
# from tensorflow.python.keras.layers.convolutional import Conv2D , MaxPooling2D
# from tensorflow.python.keras.layers.convolutional import MaxPooling2D
# from tensorflow.python.keras.layers import Activation , Flatten, Dropout, Dense
# from tensorflow.python.keras import backend as K

In [12]:
import tensorflow as tf

model = tf.python.keras.models.Sequential([
    
    tf.python.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(226, 226, 3)),
    tf.python.keras.layers.MaxPooling2D(2, 2),
    
    tf.python.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.python.keras.layers.MaxPooling2D(2,2),
    
    tf.python.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.python.keras.layers.MaxPooling2D(2,2),
    
    tf.python.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.python.keras.layers.MaxPooling2D(2,2),
    
    tf.python.keras.layers.Flatten(),
    tf.python.keras.layers.Dense(512, activation='relu'),
    
    tf.python.keras.layers.Dense(3, activation='softmax')
])

Instructions for updating:
Colocations handled automatically by placer.


In [13]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 224, 224, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 110, 110, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 55, 55, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 53, 53, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 26, 26, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 24, 24, 128)       147584    
__________

In [39]:
import tensorflow as tf

C2= tf.python.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(226, 226, 3))
C3= tf.python.keras.layers.Conv2D(64, (3,3), activation='relu')
C4= tf.python.keras.layers.Conv2D(128, (3,3), activation='relu')






model = tf.python.keras.models.Sequential([
    
#     tf.python.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(226, 226, 3)),
#     tf.python.keras.layers.MaxPooling2D(2, 2),
    
#     tf.python.keras.layers.Conv2D(64, (3,3), activation='relu'),
#     tf.python.keras.layers.MaxPooling2D(2,2),
    
#     tf.python.keras.layers.Conv2D(128, (3,3), activation='relu'),
#     tf.python.keras.layers.MaxPooling2D(2,2),
    
#     tf.python.keras.layers.Conv2D(128, (3,3), activation='relu'),
#     tf.python.keras.layers.MaxPooling2D(2,2),
    
    fpn((3,3),C2,C3,C4)
    
#     tf.python.keras.layers.Flatten(),
#     tf.python.keras.layers.Dense(512, activation='relu'),
    
#     tf.python.keras.layers.Dense(3, activation='softmax')
])

TypeError: unsupported operand type(s) for %: 'tuple' and 'int'

# SPP

In [18]:

import torch
import torch.nn.functional as F
import math
#math.ceil() method rounds a number upwards to the nearest integer, and returns the result
def spatial_pyramid_pool(self,previous_conv,num_sample,previous_conv_size,out_pool_size):
    '''
    previous_conv: a tensor vector of previous convolution layer
    num_sample: an int number of image in the batch
    previous_conv_size: an int vector [height, width] of the matrix features size of previous convolution layer
    out_pool_size: a int vector of expected output size of max pooling layer
    
    returns: a tensor vector with shape [1 x n] is the concentration of multi-level pooling
    '''    
    #print (previous_conv_size)
    for i in range(len(out_pool_size)):
        h_wid=int(math.ceil(previous_conv_size[0]/out_pool_size[i]))
        w_wid=int(math.seil(previous_conv_size[1]/out_pool_size[i]))
        h_pad=(h_wid*out_pool_size[i]-previous_conv_size[0]+1)/2
        w_pad=(w_wid*out_pool_size[i]-previous_conv_size[1]+1)/2
        maxpool=nn.MaxPool2d((h_wid,w_wid),stride=(h_wid,w_wid),padding=(h_pad,w_pad))
        x = maxpool(previous_conv)
        if(i == 0):
            spp = x.view(num_sample,-1)
            # print("spp size:",spp.size())
        else:
            # print("size:",spp.size())
            spp = torch.cat((spp,x.view(num_sample,-1)), 1)
    return spp

In [19]:
import torch
import torch.nn as nn
from torch.nn import init
import functools
from torch.autograd import Variable
import numpy as np
import torch.nn.functional as F
class SPP_NET(nn.Module):
    '''
    A CNN model which adds spp layer so that we can input multi-size tensor
    '''
    def __init__(self, opt, input_nc, ndf=64,  gpu_ids=[]):
        super(SPP_NET, self).__init__()
        self.gpu_ids = gpu_ids
        self.output_num = [4,2,1]
        
        self.conv1 = nn.Conv2d(input_nc, ndf, 4, 2, 1, bias=False)
        
        self.conv2 = nn.Conv2d(ndf, ndf * 2, 4, 1, 1, bias=False)
        self.BN1 = nn.BatchNorm2d(ndf * 2)

        self.conv3 = nn.Conv2d(ndf * 2, ndf * 4, 4, 1, 1, bias=False)
        self.BN2 = nn.BatchNorm2d(ndf * 4)

        self.conv4 = nn.Conv2d(ndf * 4, ndf * 8, 4, 1, 1, bias=False)
        self.BN3 = nn.BatchNorm2d(ndf * 8)

        self.conv5 = nn.Conv2d(ndf * 8, 64, 4, 1, 0, bias=False)
        self.fc1 = nn.Linear(10752,4096)
        self.fc2 = nn.Linear(4096,1000)

    def forward(self,x):
        x = self.conv1(x)
        x = self.LReLU1(x)

        x = self.conv2(x)
        x = F.leaky_relu(self.BN1(x))

        x = self.conv3(x)
        x = F.leaky_relu(self.BN2(x))
        
        x = self.conv4(x)
        # x = F.leaky_relu(self.BN3(x))
        # x = self.conv5(x)
        spp = spatial_pyramid_pool(x,1,[int(x.size(2)),int(x.size(3))],self.output_num)
        # print(spp.size())
        fc1 = self.fc1(spp)
        fc2 = self.fc2(fc1)
        s = nn.Sigmoid()
        output = s(fc2)
        return output
    
    
if __name__ == '__main__':
    cfg = {'FPN': True, 'feature_channel_number':256, 'backbone_type':'R50'}
    bb = FPN(cfg)
    x = torch.randn(1, 3, 1024, 1024)
    y = bb(x)
    for i in y:
        print(i.size())

In [20]:
SPP_NET

__main__.SPP_NET